## project : MBTI classification with LIWC data / skku 기계학습 수업
info : MBTI를 공개한 사람들이 올린 텍스트의 LIWC(Linguistic Inquiry and Word Count) 분석값을 바탕으로
T(사고형)과 F(감정형)을 분류하는 프로젝트 
name : 조병웅  
model : 해당 프로젝트에는 logistic 회귀 scratch 코드 모델을 사용.  


## 캐글 데이터 로딩 및 라이브러리 호출
캐글 주소 -> https://www.kaggle.com/competitions/skku-2023-1-machine-learning-second-project/overview

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## data 전처리 부분

In [7]:
data = pd.read_csv('/content/drive/MyDrive/train.csv')

In [8]:
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [9]:
test

,id,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0,1288,38.60,40.80,64.89,31.05,21.83,18.09,82.92,51.01,...,7.30,2.02,0.00,1.16,0.47,0.39,0.00,3.11,0.16,15.99
1,1,953,21.92,23.14,49.21,34.90,16.72,15.22,76.39,47.85,...,3.78,0.63,0.00,1.05,1.36,1.05,0.00,5.56,2.31,18.26
2,2,1388,36.27,26.30,85.07,62.65,13.35,15.06,83.93,51.59,...,6.34,0.72,0.00,0.58,0.22,0.79,0.00,3.75,0.36,12.10
3,3,1294,42.36,60.67,26.94,99.00,23.53,15.69,84.31,49.77,...,4.25,2.78,1.24,0.77,1.39,0.85,0.00,3.63,0.77,15.53
4,4,1377,19.83,49.42,53.79,53.05,105.92,14.16,88.24,57.52,...,0.29,0.58,0.00,0.22,2.54,0.65,0.00,0.80,0.73,11.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,1730,797,47.84,33.93,72.52,61.46,15.33,15.31,86.20,51.94,...,4.02,0.50,0.00,1.38,0.38,1.13,0.00,0.75,0.50,16.94
1731,1731,1231,37.88,24.23,76.55,31.30,21.98,19.25,84.81,53.94,...,3.74,0.49,0.08,0.73,0.08,1.22,0.08,3.98,1.71,13.24
1732,1732,996,51.62,50.80,51.90,46.09,23.16,16.77,81.12,49.50,...,0.70,0.00,0.00,0.90,0.60,2.41,0.00,2.51,0.00,20.18
1733,1733,1610,19.15,23.00,79.30,91.92,21.76,15.90,84.78,54.10,...,4.72,1.49,0.00,0.68,2.98,0.99,0.00,6.40,1.55,10.99


In [10]:
data

,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,...,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP,label
0,1078,51.81,51.86,38.80,55.50,25.07,21.80,74.86,43.88,14.66,...,0.37,0.19,1.48,0.00,1.95,0.0,3.62,1.67,19.20,0
1,883,12.54,52.27,60.12,88.84,19.20,14.04,92.07,60.82,20.84,...,0.34,0.00,1.70,0.23,0.91,0.0,6.12,0.45,19.71,0
2,1593,63.46,62.10,37.47,57.83,28.96,14.69,83.18,57.06,17.01,...,0.06,0.00,0.13,0.06,1.00,0.0,2.20,0.00,10.48,0
3,970,36.80,24.16,72.78,92.82,13.29,14.74,91.75,59.07,19.59,...,2.27,0.00,0.82,1.44,0.10,0.0,4.43,0.41,11.44,1
4,1339,39.78,47.91,38.15,95.89,28.49,17.63,86.41,52.73,17.48,...,5.30,0.07,1.42,1.49,1.79,0.0,1.57,0.97,12.99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6935,1459,37.27,13.04,94.70,88.49,44.21,14.05,86.36,54.83,17.96,...,0.96,0.00,0.55,0.34,1.64,0.0,2.40,2.26,11.79,1
6936,1654,22.17,16.36,87.19,64.73,23.63,14.51,90.57,61.91,20.19,...,0.24,0.00,0.18,0.12,0.42,0.0,6.41,0.12,9.67,1
6937,1673,42.46,30.78,85.61,76.57,15.35,17.27,87.99,57.14,19.13,...,0.24,0.06,0.24,1.49,0.30,0.0,2.39,0.66,9.38,1
6938,1433,25.87,55.54,30.88,95.40,11.65,15.00,88.35,55.97,17.10,...,0.21,0.07,1.05,2.58,0.70,0.0,2.37,0.98,11.30,1


In [12]:
#이상치 제거
for i in data.columns:
  if i == 'label':
    pass
  else:
    p25 = data[i].quantile(0.25)
    p75 = data[i].quantile(0.75)
    iqr = p75- p25 
    m = p25 - 1.5*iqr
    M = p75 + 1.5*iqr
    data[i] = np.where((data[i] < m)| (data[i] > M), np.nan, data[i])
    data = data.dropna(axis = 0)

<ipython-input-12-761b05010f69>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = np.where((data[i] < m)| (data[i] > M), np.nan, data[i])
<ipython-input-12-761b05010f69>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = np.where((data[i] < m)| (data[i] > M), np.nan, data[i])
<ipython-input-12-761b05010f69>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

## 모델

In [14]:
#로지스틱 회귀 참고.
__author__ = "조병웅"
__id__ = "2019312570"

import numpy as np
class LogisticRegression:
    def __init__(self, max_iter=500, penalty="l2", initialize = "one", random_seed = 1213):
        """
        이 함수는 수정할 필요가 없습니다.
        """
        self.author = __author__
        self.id = __id__
        
        self.max_iter = max_iter
        self.penalty = penalty
        self.initialize = initialize
        self.random_seed = random_seed
        self.lr = 0.01
        self.lamb = 0.01
        np.random.seed(self.random_seed)

        if self.penalty not in ["l1", "l2"]:
            raise ValueError("Penalty must be l1 or l2")

        if self.initialize not in ["one", "LeCun", "random"]:
            raise ValueError("Only [LeCun, One, random] Initialization supported")

            
    def activation(self, z):

        a = 1/(1+np.exp(-z))
        return a


    def fwpass(self, x):

        z = np.dot(x, self.w) + self.b

        z = self.activation(z)              
        return z


    def bwpass(self, x, err): 

        if self.penalty == "l1":
            sum = 0 
            for i in range(len(self.w)):
                sum += abs(self.w[i]) 
            w_grad = (2/len(x))*(np.dot(err,x)) + self.lamb*sum 
        elif self.penalty == "l2":
            sum = 0 
            for i in range(len(self.w)):
                sum += self.w[i]**2 
            w_grad = (2/len(x))*(np.dot(err,x)) + self.lamb*sum 
        
        b_grad = (2/len(x))*np.sum(err) 

        return w_grad, b_grad

    
    def initialize_w(self, x):

        w_library = {
            "one":np.ones(x.shape[1]),
            "LeCun":np.random.uniform(low = -np.sqrt(1.0 / x.shape[1]), high = np.sqrt(1.0 / x.shape[1]), size = x.shape[1]),  
            "random":np.random.randint(0,1, size = x.shape[1]) 
        }

        return w_library[self.initialize]


    def fit(self, x, y):
        self.w = self.initialize_w(x)
        self.b = 0
        for _ in range(self.max_iter):
            z = self.fwpass(x)
            err = -(y - z) 
            w_grad, b_grad = self.bwpass(x, err)

            self.w = self.w - self.lr*w_grad 


            self.b = self.b - self.lr*self.b 

        return self.w, self.b

#csv에 넣기 위한 부분
    def predict(self, x):

        z = self.fwpass(x)
        for i in range(len(z)):
            if z[i] > 0.5: 
                z[i] = 1
            else :
                z[i] = 0
        return z

#csv 파일 만들기 위해 추가된 부분
    def make_sub(self, x):
        z = self.predict(x)
        z = z.astype(np.int64)
        sub_df = pd.DataFrame({'label' : z})
        sub_df.index = range(0,z.shape[0])
        sub_df.to_csv('submission_example.csv', index_label=['id'])

    def score(self, x, y):

        return np.mean(self.predict(x) == y)
    
    
    def feature_importance(self, coef, column_to_use):

        w = self.w.copy()
        for i in range(len(self.w)): 
            w[i] = abs(self.w[i]) 
        dic = {} 
        for i in range(len(w)):
            dic[w[i]] = column_to_use[i]
        s = w.argsort() 
        w = w[s]
        self.w = self.w[s]   
        rank_li = [] 
        for i in range(len(column_to_use)):
            rank_li.append(((len(w)-(i+1)), [dic[w[i]], self.w[i]])) 
        rank_li.reverse()
        return rank_li

In [17]:
#피처로 사용할 컬럼의 수.
column_to_use = ['WC','Analytic', 'Clout', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'Dic',
       'function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they',
       'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate',
       'verb', 'adj', 'compare', 'interrog', 'number', 'quant', 'affect',
       'posemo', 'negemo', 'anx', 'anger', 'sad', 'social', 'family', 'friend',
       'female', 'male', 'cogproc', 'insight', 'cause', 'discrep', 'tentat',
       'certain', 'differ', 'percept', 'see', 'hear', 'feel', 'bio', 'body',
       'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve',
       'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture',
       'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home',
       'money', 'relig', 'death', 'informal', 'swear', 'netspeak', 'assent',
       'nonflu', 'filler', 'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC',
       'QMark', 'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP']
train_x = data[column_to_use].copy()
train_y = data['label'].copy()

In [18]:
#비교분석 위해 사용한 k폴드. 하지만 피처랑 파라미터의 잘못인지 정확도가 더 낮게 나옴. / reject함.
"""
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import time

ss=StandardScaler()
#ss.fit(train_x)
#train_x = ss.transform(train_x)

kfold = KFold(n_splits=5, shuffle = True)
cv_accuracy = []
n_iter = 0

lr = LogisticRegression(max_iter=700, penalty="l2", initialize = "LeCun")
start = time.time()

for train_index, test_index in kfold.split(data):  
    x_train, x_test = train_x.iloc[train_index], train_x.iloc[test_index]
    y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]

    x_train=ss.fit_transform(x_train)
    x_test=ss.transform(x_test) 

    coef, interrupt = lr.fit(x_train, y_train)
    pred = lr.predict(x_test)
    n_iter += 1

    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))
#shuffle 있으면 0.74 / 없으면 0.75
#람다 1일때, 100 72, 700 74, 900 74
#람다 2 / 300 73 500 74  700 72 900 67
"""

'\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.model_selection import KFold\nfrom sklearn.metrics import accuracy_score\nimport time\n\nss=StandardScaler()\n#ss.fit(train_x)\n#train_x = ss.transform(train_x)\n\nkfold = KFold(n_splits=5, shuffle = True)\ncv_accuracy = []\nn_iter = 0\n\nlr = LogisticRegression(max_iter=700, penalty="l2", initialize = "LeCun")\nstart = time.time()\n\nfor train_index, test_index in kfold.split(data):  \n    x_train, x_test = train_x.iloc[train_index], train_x.iloc[test_index]\n    y_train, y_test = train_y.iloc[train_index], train_y.iloc[test_index]\n\n    x_train=ss.fit_transform(x_train)\n    x_test=ss.transform(x_test) \n\n    coef, interrupt = lr.fit(x_train, y_train)\n    pred = lr.predict(x_test)\n    n_iter += 1\n\n    accuracy = np.round(accuracy_score(y_test, pred), 4) \n    train_size = x_train.shape[0]\n    test_size = x_test.shape[0]\n    print(\'\n#{0} 교차 검증 정확도 : {1},  학

## 결과

In [19]:
#테스트 데이터 정확도 82가 나옴. 구현부 부분. 실제 정확도랑은 꽤 차이가 있음. 과적합 발생 한것으로 보임 
import time
from  sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, random_state = 11)
ss=StandardScaler()
train_x=ss.fit_transform(train_x)
test_x=ss.transform(test_x) 

lr = LogisticRegression(max_iter=300, penalty="l2", initialize = "LeCun")
start = time.time()
coef, interrupt = lr.fit(train_x, train_y)
print(f'time: {time.time() - start}')
print(f"Accuray: {lr.score(test_x, test_y)}")

# 100 81 / 람다 0.2일때  0.809
# 300 81 / 0.82
# 500 82 / 0.80
# 700 81 / 0.79


time: 0.5095615386962891
Accuray: 0.8206106870229007


In [22]:
#피처 상관관계
lr.feature_importance(coef, column_to_use)


[(0, ['achieve', -0.30023278342867293]),
 (1, ['swear', -0.27632802334296047]),
 (2, ['work', -0.22669933849097382]),
 (3, ['cause', -0.22626673383401144]),
 (4, ['feel', 0.22425605420760036]),
 (5, ['ingest', -0.21733114284452773]),
 (6, ['OtherP', -0.20800481578614358]),
 (7, ['interrog', -0.19738669680765802]),
 (8, ['article', -0.19475860781118168]),
 (9, ['netspeak', 0.18239923317887333]),
 (10, ['negate', -0.1806686669125388]),
 (11, ['Exclam', 0.174008172110954]),
 (12, ['percept', 0.16593638499019858]),
 (13, ['anger', -0.16539838809639498]),
 (14, ['Sixltr', -0.15745640118923804]),
 (15, ['health', 0.1423573367655718]),
 (16, ['anx', 0.1355303289049337]),
 (17, ['bio', 0.13283222246272305]),
 (18, ['affect', 0.12415924314087332]),
 (19, ['time', -0.1230042910675488]),
 (20, ['Period', -0.11882171612564317]),
 (21, ['quant', -0.11831058531660578]),
 (22, ['focusfuture', -0.11669344847912337]),
 (23, ['Authentic', 0.11167941590734333]),
 (24, ['ppron', 0.11039931029268903]),
 (2

In [24]:
test_data = test[column_to_use].copy()
test_data = ss.transform(test_data)

In [25]:
lr.predict(test_data)

array([1., 0., 1., ..., 0., 0., 0.])

In [26]:
lr.make_sub(test_data)


In [27]:
!kaggle competitions submit -c skku-2023-1-machine-learning-second-project -f submission_example.csv -m "Message"

100% 10.8k/10.8k [00:00<00:00, 57.3kB/s]
Successfully submitted to [SKKU 2023-1 Machine Learning] Second Project